In [19]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
import os
import cv2
import numpy as np

In [21]:
pasta_imagens = "/content/drive/MyDrive/Visão Computacional/TrabalhoFinal/frutas"
pastas = ['acerola', 'abacate', 'carambola', 'coco', 'jabuticaba']

descritores = []

for pasta in pastas:
    caminho_pasta = pasta_imagens + '/' + pasta
    arquivos_imagem = os.listdir(caminho_pasta)
    
    if len(arquivos_imagem) == 0:
        print("A pasta {} está vazia. Continuando para a próxima pasta.".format(pasta))
        continue
    
    for arquivo_imagem in arquivos_imagem:
        caminho_imagem = caminho_pasta + '/' + arquivo_imagem
        imagem = cv2.imread(caminho_imagem)
        
        if imagem is None:
            print("Erro ao ler a imagem {}. Continuando para a próxima imagem.".format(caminho_imagem))
            continue
        
        cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
        redimensionada = cv2.resize(cinza, (64, 128))
        
        tamanho_bloco = (16, 16)
        passo_bloco = (8, 8)
        tamanho_celula = (8, 8)
        num_bins = 9

        hog = cv2.HOGDescriptor((64, 128), tamanho_bloco, passo_bloco, tamanho_celula, num_bins)

        descritor = hog.compute(redimensionada)

        descritores.append(descritor)

In [22]:
arquivo_saida = '/content/drive/MyDrive/Visão Computacional/TrabalhoFinal/frutas/descritores_frutas.txt'

pastas_processadas = set()

with open(arquivo_saida, 'w') as arquivo:
    for i in range(len(pastas)):
        pasta = pastas[i]
        descritor = descritores[i]
        rotulo = pasta.split('/')[-1]
        
        if rotulo in pastas_processadas:
            continue
        
        pastas_processadas.add(rotulo)
        
        arquivo.write("Descritor da imagem 1 da pasta {}:\n".format(rotulo))
        descricao_descritor = ','.join(map(str, descritor))
        arquivo.write(descricao_descritor + '\n\n')

In [23]:
rotulos = []
for i, pasta in enumerate(pastas):
    num_imagens = len(os.listdir(os.path.join(pasta_imagens, pasta)))
    rotulos.extend([pasta] * num_imagens)

X_treino, X_teste, y_treino, y_teste = train_test_split(descritores, rotulos, test_size=0.2, random_state=42)

svm = LinearSVC()
svm.fit(X_treino, y_treino)

y_predito = svm.predict(X_teste)

for true_label, pred_label in zip(y_teste, y_predito):
    print("Rótulo: {} -> Predito: {}".format(true_label, pred_label))

Rótulo: carambola -> Predito: carambola
Rótulo: coco -> Predito: acerola
Rótulo: coco -> Predito: coco
Rótulo: coco -> Predito: carambola
Rótulo: carambola -> Predito: acerola
Rótulo: coco -> Predito: coco
Rótulo: coco -> Predito: abacate
Rótulo: carambola -> Predito: carambola
Rótulo: jabuticaba -> Predito: acerola
Rótulo: acerola -> Predito: coco
Rótulo: jabuticaba -> Predito: acerola
Rótulo: acerola -> Predito: abacate
Rótulo: coco -> Predito: acerola
Rótulo: jabuticaba -> Predito: coco
Rótulo: jabuticaba -> Predito: acerola
Rótulo: jabuticaba -> Predito: acerola
Rótulo: abacate -> Predito: abacate
Rótulo: jabuticaba -> Predito: jabuticaba
Rótulo: coco -> Predito: coco
Rótulo: abacate -> Predito: abacate
Rótulo: carambola -> Predito: carambola
Rótulo: acerola -> Predito: abacate
Rótulo: abacate -> Predito: abacate
Rótulo: carambola -> Predito: acerola
Rótulo: abacate -> Predito: abacate
Rótulo: abacate -> Predito: coco
Rótulo: acerola -> Predito: acerola
Rótulo: jabuticaba -> Predit

In [24]:
accuracy = accuracy_score(y_teste, y_predito)
print(f"Acurácia (%): {accuracy * 100:.2f}%")

precision = precision_score(y_teste, y_predito, average='weighted')
print(f"Precisão (%): {precision * 100:.2f}%")

confusion_mat = confusion_matrix(y_teste, y_predito)
print("Matriz de Confusão:")
print(confusion_mat)

Acurácia (%): 47.50%
Precisão (%): 48.80%
Matriz de Confusão:
[[25  4  6  7  6]
 [ 5 11  3  9  5]
 [ 4  6 17  6  4]
 [ 3  7  3 18  7]
 [ 5 10  1  4 24]]
